In [ ]:
!pip install evalml

In [ ]:
!pip install --upgrade dask

In [74]:

# readinng csv file
import pandas as pd 
data = pd.read_csv('hour.csv') 

data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [75]:
data= data.drop(['instant','dteday'],axis=1)

In [76]:
# Seperating our dataset into Independent And Dependent Features 
X=data.drop('cnt',axis=1) 
y=data['cnt']

In [77]:
# Check whether dataset is balanced or not
y.value_counts(normalize=True)

5      0.014961
6      0.013580
4      0.013292
3      0.012889
2      0.011968
         ...   
725    0.000058
709    0.000058
661    0.000058
629    0.000058
887    0.000058
Name: cnt, Length: 869, dtype: float64

In [96]:
import evalml
# split dataset into training and testing part using evalml, here we have to mention problem_type, what kind of problem actually we are trying to solve. 
X_train,X_test,y_train,y_test = evalml.preprocessing.split_data(X,y,problem_type='regression',test_size=.3)

In [97]:
# printing all problem statement available in evalml library.
evalml.problem_types.ProblemTypes.all_problem_types

[<ProblemTypes.BINARY: 'binary'>,
 <ProblemTypes.MULTICLASS: 'multiclass'>,
 <ProblemTypes.REGRESSION: 'regression'>,
 <ProblemTypes.TIME_SERIES_REGRESSION: 'time series regression'>,
 <ProblemTypes.TIME_SERIES_BINARY: 'time series binary'>,
 <ProblemTypes.TIME_SERIES_MULTICLASS: 'time series multiclass'>]

In [110]:
# Search best suitable algorithm for our problem statement
from evalml.automl import AutoMLSearch
%timeit 
automl = AutoMLSearch(X_train=X_train,y_train=y_train,problem_type='regression',max_batches=1,optimize_thresholds=True)
automl.search()

Generating pipelines to search over...

*****************************
* Beginning pipeline search *
*****************************

Optimizing for R2. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: random_forest, xgboost, linear_model, catboost, decision_tree, lightgbm, extra_trees



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mean Baseline Regression Pipeline
Mean Baseline Regression Pipeline:
	Starting cross validation
	Finished cross validation - mean R2: -0.000
	High coefficient of variation (cv >= 0.2) within cross validation scores.
	Mean Baseline Regression Pipeline may not perform as estimated on unseen data.

*****************************
* Evaluating Batch Number 1 *
*****************************

Decision Tree Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.991
[04:46:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:46:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:46:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2:

In [104]:
# return a table showing different models based on their ranking
automl.rankings

,id,pipeline_name,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,4,Linear Regressor w/ Imputer + Standard Scaler,1.000000,0.000000,1.000000,371216.087457,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,2,XGBoost Regressor w/ Imputer,0.999692,0.000038,0.999721,371101.640401,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,3,Random Forest Regressor w/ Imputer,0.997176,0.000518,0.996585,370167.886937,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,5,Extra Trees Regressor w/ Imputer,0.995023,0.000696,0.995267,369369.148268,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,1,Decision Tree Regressor w/ Imputer,0.991171,0.000261,0.991176,367939.572394,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,6,LightGBM Regressor w/ Imputer,0.983065,0.000211,0.982983,364931.358674,False,{'Imputer': {'categorical_impute_strategy': 'm...
6,8,Elastic Net Regressor w/ Imputer + Standard Sc...,0.967918,0.000444,0.967568,359309.921902,False,{'Imputer': {'categorical_impute_strategy': 'm...
7,7,CatBoost Regressor w/ Imputer,0.414396,0.004711,0.410043,153889.169329,False,{'Imputer': {'categorical_impute_strategy': 'm...
8,0,Mean Baseline Regression Pipeline,-0.000269,0.000229,-0.000502,0.000000,True,{'Baseline Regressor': {'strategy': 'mean'}}


In [100]:
# get best pipeline
best_pipeline = automl.best_pipeline

# Output : 
# GeneratedPipeline(parameters={'Random Forest Classifier':{'n_estimators': 100, 'max_depth': 6, 'n_jobs': -1},})

In [101]:
best_pipeline

GeneratedPipeline(parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'Linear Regressor':{'fit_intercept': True, 'normalize': False, 'n_jobs': -1},})

In [102]:
# describe best pipeline in more detail.
automl.describe_pipeline(automl.rankings.iloc[0]["id"])


*************************************************
* Linear Regressor w/ Imputer + Standard Scaler *
*************************************************

Problem Type: regression
Model Family: Linear

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. Standard Scaler
3. Linear Regressor
	 * fit_intercept : True
	 * normalize : False
	 * n_jobs : -1

Training
Training for regression problems.
Total training time (including CV): 0.9 seconds

Cross Validation
----------------
               R2  ExpVariance  MaxError  MedianAE   MSE   MAE  Root Mean Squared Error # Training # Validation
0           1.000        1.000     0.000     0.000 0.000 0.000                    0.000      8,110        4,055
1           1.000        1.000     0.000     0.000 0.000 0.000                    0.000      8,110        4,055
2           1.000        1.000     0.000     0.000 0.000 0.000 

In [103]:
# evalute test data
scores = best_pipeline.score(X_test, y_test,  objectives=evalml.objectives.get_core_objectives('regression')) 
print(f'Accuracy regression: {scores["MAE"]:.10f}')


Accuracy regression: 0.0000000000


In [92]:
scores

OrderedDict([('ExpVariance', 1.0),
             ('MaxError', 6.821210263296962e-13),
             ('MedianAE', 8.526512829121202e-14),
             ('MSE', 1.92634713175218e-26),
             ('MAE', 1.0324946370322974e-13),
             ('R2', 1.0),
             ('Root Mean Squared Error', 1.387929080231472e-13)])

In [108]:
pipeline = automl.best_pipeline
#new = pipeline.predict(X_test)
prediction = pipeline.predict(X_test)

output = prediction.to_series()
output

0         7.0
1         5.0
2       743.0
3       208.0
4       333.0
        ...  
5209     35.0
5210    571.0
5211    499.0
5212    302.0
5213    229.0
Name: cnt, Length: 5214, dtype: float64

In [105]:
type(prediction)

woodwork.datacolumn.DataColumn

In [89]:
print("test length",len(y_test.to_series()))
print("prediction length",len(output))

test length 3476
prediction length 3476


In [90]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test.to_series(), output)
#cm = ConfusionMatrix(y_true, y_predicted)
cm


ValueError: ignored

In [109]:
from sklearn.metrics import r2_score

r2_score(y_test.to_series(), output)

1.0

In [ ]:
# save the model in model.pkl file.
best_pipeline.save("model.pkl")